## Load Data

In [99]:
import pandas as pd
import numpy as np
import pickle
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from scipy import stats

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [101]:
def load_wesad_subject(subject_path):
    """Loads a WESAD subject's .pkl file and extracts CHEST data."""
    with open(subject_path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')

    # Extract Chest Data (700 Hz)
    chest_data = data['signal']['chest']
    
    # Create DataFrame
    df = pd.DataFrame()
    df['ACC_x'] = chest_data['ACC'][:, 0]
    df['ACC_y'] = chest_data['ACC'][:, 1]
    df['ACC_z'] = chest_data['ACC'][:, 2]
    df['ECG']   = chest_data['ECG'].flatten()
    df['EMG']   = chest_data['EMG'].flatten()
    df['EDA']   = chest_data['EDA'].flatten()
    df['Temp']  = chest_data['Temp'].flatten()
    df['Resp']  = chest_data['Resp'].flatten()
    
    # Add Labels & Subject ID
    df['label'] = data['label']
    df['subject'] = data['subject']
    
    return df

def create_windows(df, window_seconds=2, stride_seconds=0.25, sampling_rate=700):
    """Slices the dataframe into overlapping windows."""
    window_steps = int(window_seconds * sampling_rate)
    stride_steps = int(stride_seconds * sampling_rate)
    
    feature_cols = ['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
    data = df[feature_cols].values
    labels = df['label'].values
    
    X_windows = []
    y_windows = []
    
    for i in range(0, len(df) - window_steps, stride_steps):
        window_data = data[i : i + window_steps]
        window_labels = labels[i : i + window_steps]
        
        # Take the most frequent label in this window
        mode_label = stats.mode(window_labels, keepdims=True)[0][0]
        
        X_windows.append(window_data.transpose()) 
        y_windows.append(mode_label)
        
    return np.array(X_windows), np.array(y_windows)

In [102]:
# --- CONFIGURATION ---
base_path = r"H:\Research\archive (1)\WESAD"  # Your specific path
subject_id = "S2"
file_path = os.path.join(base_path, subject_id, f"{subject_id}.pkl")

if os.path.exists(file_path):
    print(f"Loading {subject_id}...")
    df = load_wesad_subject(file_path)

    # 1. Filter Labels (1=Baseline, 2=Stress)
    df = df[df['label'].isin([1, 2])].copy()
    
    # 2. Remap Labels to 0 (Baseline) and 1 (Stress)
    label_map = {1: 0, 2: 1}
    df['label'] = df['label'].map(label_map)
    
    # 3. Create Windows
    print("Creating windows...")
    X, y = create_windows(df, window_seconds=2, stride_seconds=0.25)
    print(f"Original Window shape: {X.shape}") 

    # 4. Normalize (StandardScaler) - CRITICAL STEP
    N, C, T = X.shape
    X_flat = X.transpose(0, 2, 1).reshape(-1, C) # Flatten for scaler
    scaler = StandardScaler()
    X_scaled_flat = scaler.fit_transform(X_flat)
    X_norm = X_scaled_flat.reshape(N, T, C).transpose(0, 2, 1) # Reshape back
    
    print(f"Data Normalized. Mean: {X_norm.mean():.2f}, Std: {X_norm.std():.2f}")

    # 5. Time-Based Split (No Shuffling!)
    # We use X_norm (the clean data), NOT X_final (the raw data)
    tensor_x = torch.Tensor(X_norm)
    tensor_y = torch.Tensor(y).long()

    split_idx = int(0.8 * len(tensor_x))

    X_train = tensor_x[:split_idx]
    y_train = tensor_y[:split_idx]
    X_test = tensor_x[split_idx:]
    y_test = tensor_y[split_idx:]

    # 6. Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

    print(f"Ready for Training! Train samples: {len(X_train)} | Test samples: {len(X_test)}")

else:
    print(f"Error: File not found at {file_path}")

Loading S2...
Creating windows...
Creating windows...
Original Window shape: (7028, 8, 1400)
Original Window shape: (7028, 8, 1400)
Data Normalized. Mean: -0.00, Std: 1.00
Ready for Training! Train samples: 5622 | Test samples: 1406
Data Normalized. Mean: -0.00, Std: 1.00
Ready for Training! Train samples: 5622 | Test samples: 1406


In [103]:
class LightweightModel(nn.Module):
    def __init__(self, input_channels=8, num_classes=2, window_size=1400):
        super(LightweightModel, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        # Classifier
        final_dim = window_size // 4 
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * final_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

model = LightweightModel().to(device)
print("Model initialized.")

Model initialized.


In [104]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 10

print("Starting Training...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Epoch {epoch+1} | Loss: {running_loss/len(train_loader):.4f} | Acc: {100*correct/total:.2f}%")

print("Training Complete.")

Starting Training...
Epoch 1 | Loss: 0.5667 | Acc: 97.39%
Epoch 1 | Loss: 0.5667 | Acc: 97.39%
Epoch 2 | Loss: 0.4009 | Acc: 93.33%
Epoch 2 | Loss: 0.4009 | Acc: 93.33%
Epoch 3 | Loss: 0.3735 | Acc: 92.28%
Epoch 3 | Loss: 0.3735 | Acc: 92.28%
Epoch 4 | Loss: 0.0606 | Acc: 98.70%
Epoch 4 | Loss: 0.0606 | Acc: 98.70%
Epoch 5 | Loss: 0.0483 | Acc: 99.52%
Epoch 5 | Loss: 0.0483 | Acc: 99.52%
Epoch 6 | Loss: 0.0396 | Acc: 99.57%
Epoch 6 | Loss: 0.0396 | Acc: 99.57%
Epoch 7 | Loss: 0.0296 | Acc: 99.54%
Epoch 7 | Loss: 0.0296 | Acc: 99.54%
Epoch 8 | Loss: 0.0713 | Acc: 98.42%
Epoch 8 | Loss: 0.0713 | Acc: 98.42%
Epoch 9 | Loss: 0.0898 | Acc: 99.50%
Epoch 9 | Loss: 0.0898 | Acc: 99.50%
Epoch 10 | Loss: 0.0300 | Acc: 99.57%
Training Complete.
Epoch 10 | Loss: 0.0300 | Acc: 99.57%
Training Complete.


In [105]:
def test_model_under_attack(model, loader, noise_level=0.0):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Attack: Add noise
            noise = torch.randn_like(inputs) * noise_level
            attacked_inputs = inputs + noise
            
            outputs = model(attacked_inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return 100 * correct / total

print(f"Clean Acc: {test_model_under_attack(model, test_loader, 0.0):.2f}%")
print(f"Attack (0.5): {test_model_under_attack(model, test_loader, 0.5):.2f}%")
print(f"Attack (2.0): {test_model_under_attack(model, test_loader, 2.0):.2f}%")

Clean Acc: 65.58%
Attack (0.5): 63.02%
Attack (0.5): 63.02%
Attack (2.0): 12.73%
Attack (2.0): 12.73%
